In [6]:
%display latex
from delierium import *
from pprint import pprint 
from collections.abc import Iterable
import functools
from operator import mul, sub
from pprint import pprint
from IPython.core.debugger import set_trace

In [21]:
var ("x1 x2 x3")
print (x1.__class__)
m = ((x1**2)*(x2**2)*(x3**3), (x1**3)*(x3**3), (x1**3)*x2*x3, x2*x3)

<class 'sage.symbolic.expression.Expression'>


In [35]:
print(m [0], "::", multipliers (m[0], m, (x3,x2,x1)))
print(m [1], "::", multipliers (m[1], m, (x3,x2,x1)))
print(m [2], "::", multipliers (m[2], m, (x3,x2,x1)))
print(m [3], "::", multipliers (m[3], m, (x3,x2,x1)))

x1^2*x2^2*x3^3 :: ([x3, x2, x1], set())
x1^3*x3^3 :: ([x3, x1], {x2})
x1^3*x2*x3 :: ([x2, x1], {x3})
x2*x3 :: ([x2], {x3, x1})


In [36]:
m = ((2,2,3), (3,0,3), (3,1,1), (0,1,1))
print(m [0], "::", vec_multipliers (m[0], m, (2,1,0)))
print(m [1], "::", vec_multipliers (m[1], m, (2,1,0)))
print(m [2], "::", vec_multipliers (m[2], m, (2,1,0)))
print(m [3], "::", vec_multipliers (m[3], m, (2,1,0)))

(2, 2, 3) :: ([2, 1, 0], set())
(3, 0, 3) :: ([2, 0], {1})
(3, 1, 1) :: ([1, 0], {2})
(0, 1, 1) :: ([1], {0, 2})


In [37]:
ctx = Context([], [x3,x2,x1], Mgrevlex)

In [38]:
# now for completion
s1=[m[0], m[1]]
s2=[m[2], m[3]]
inter = []
for _m in s1:
    mul, nonmul = vec_multipliers (_m, m, (2,1,0))
    for nm in nonmul:
        hungi = list(_m)
        hungi[nm] += 1
        inter.append(tuple(hungi))
s1.extend(inter)
inter = []
for _m in s2:
    mul, nonmul = vec_multipliers (_m, m, (2,1,0))
    for nm in nonmul:
        hungi = list(_m)
        hungi[nm] += 1
        inter.append(tuple(hungi))
s2.extend(inter)

In [39]:
s1

[(2, 2, 3), (3, 0, 3), (3, 1, 3)]

In [40]:
ss1 = Reorder (s1,ctx, ascending = False); ss1

[(2, 2, 3), (3, 0, 3), (3, 1, 3)]

In [41]:
s2

[(3, 1, 1), (0, 1, 1), (3, 1, 2), (1, 1, 1), (0, 1, 2)]

In [42]:
ss2 = Reorder (s2,ctx, ascending = False)

In [43]:
print (ss2)

[(0, 1, 1), (0, 1, 2), (1, 1, 1), (3, 1, 1), (3, 1, 2)]


In [44]:
ctx_lex = Context ([], (x1,x2,x3))
ctx_grlex = Context ([], (x1,x2,x3),Mgrlex)
ctx_grevlex = Context ([], (x1,x2,x3),Mgrevlex)

In [45]:
m

((2, 2, 3), (3, 0, 3), (3, 1, 1), (0, 1, 1))

In [46]:
print (Reorder (m, ctx_lex))

[(0, 1, 1), (2, 2, 3), (3, 0, 3), (3, 1, 1)]


In [47]:
print (Reorder (m, ctx_grlex))

[(0, 1, 1), (2, 2, 3), (3, 0, 3), (3, 1, 1)]


In [48]:
print (Reorder (m, ctx_grevlex))

[(0, 1, 1), (2, 2, 3), (3, 0, 3), (3, 1, 1)]


In [49]:
lulu = Mgrevlex ((), (1,2,3))

In [50]:
print(lulu)

[ 1  1  1]
[ 0  0  0]
[ 0  0 -1]
[ 0 -1  0]
[-1  0  0]


In [51]:
grev = [[1,1,1], [0,0,-1],[0,-1,0]]

In [52]:
lulu = Mgrlex ((), (1,2,3))

In [53]:
print(lulu)

[1 1 1]
[0 0 0]
[1 0 0]
[0 1 0]
[0 0 1]


In [54]:
gr = [[1,1,1], [0,1,0],[1,0,0]]

In [55]:

gr

[[1, 1, 1], [0, 1, 0], [1, 0, 0]]

In [56]:
var ('x y z t')
ctx1 = Context ([], (x,y,z,t)) 
ctx2 = Context ([], (x,y,z,t), Mgrlex)
ctx3 = Context ([], (x,y,z,t), Mgrevlex)

In [57]:
Differential_Vector ((0,0,0,0), ctx = ctx1)

In [58]:
a1 = Differential_Vector ((0,0,0,0), ctx = ctx2)
a2 = Differential_Vector ((0,1,0,0), ctx = ctx2)

In [59]:
Differential_Vector ((0,0,0,0), ctx = ctx3)

In [60]:
a1<a2

False

In [61]:
m1 = sorted(map (lambda _ : Differential_Vector(_, ctx1), ((2, 2 ,3), (3, 0, 3), (3,1,1), (0,1,1))))
print ([_._e for _ in m1])

[(3, 1, 1), (3, 0, 3), (2, 2, 3), (0, 1, 1)]


In [62]:
m2 = reversed(sorted(map (lambda _ : Differential_Vector(_, ctx2), ((2, 2 ,3), (3, 0, 3), (3,1,1), (0,1,1)))))
print ([_._e for _ in m2])

[(0, 1, 1), (3, 1, 1), (3, 0, 3), (2, 2, 3)]


In [63]:
m3 = sorted(map (lambda _ : Differential_Vector(_, ctx3), ((2, 2 ,3), (3, 0, 3), (3,1,1), (0,1,1))))
print ([_._e for _ in m3])

[(2, 2, 3), (3, 0, 3), (3, 1, 1), (0, 1, 1)]


In [64]:
print(complete ([(2, 2 ,3), (3, 0, 3), (3,1,1), (0,1,1)], ctx_grevlex))

AttributeError: 'tuple' object has no attribute 'Lder'

In [ ]:
print(complete ([(2, 2 ,3), (3, 0, 3), (3,1,1), (0,1,1)], ctx_grlex))

In [ ]:
def integrability_condition(e1:Differential_Polynomial, e2:Differential_Polynomial):
    pass